# TREC 2018 Precision Medicine

In [1]:
import json
from json2html import *
from IPython.display import HTML
import eval

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)
#HTML(json2html.convert(json = config))

Load TOPICS and GOLD STANDARD from last year (30 topics)

In [3]:
topics_df = eval.get_topics('./topics/topics2017.xml')

In [4]:
topics_df

,topic,disease,gene,gene1,gene2,gene3,sex,age
1,1,Liposarcoma,CDK4 Amplification,CDK4 Amplification,,,male,38
2,2,Colon cancer,"KRAS (G13D), BRAF (V600E)",KRAS (G13D),BRAF (V600E),,male,52
3,3,Meningioma,"NF2 (K322), AKT1(E17K)",NF2 (K322),AKT1(E17K),,female,45
4,4,Breast cancer,"FGFR1 Amplification, PTEN (Q171)",FGFR1 Amplification,PTEN (Q171),,female,67
5,5,Melanoma,"BRAF (V600E), CDKN2A Deletion",BRAF (V600E),CDKN2A Deletion,,female,45
6,6,Melanoma,NRAS (Q61K),NRAS (Q61K),,,male,55
7,7,Lung cancer,EGFR (L858R),EGFR (L858R),,,female,50
8,8,Lung cancer,EML4-ALK Fusion transcript,EML4-ALK Fusion transcript,,,male,52
9,9,Gastrointestinal stromal tumor,KIT Exon 9 (A502_Y503dup),KIT Exon 9 (A502_Y503dup),,,female,49
10,10,Lung adenocarcinoma,KRAS (G12C),KRAS (G12C),,,female,61


In [5]:
qrels = eval.get_qrels('./gold-standard/sorted.abstracts.2017.qrels')

In [6]:
run_df = eval.run(topics_df)

RUN: FIXME {'query_template': 'baseline.json', 'disease_tie_breaker': 0.5, 'disease_boost': 1.5, 'gene_tie_breaker': 0.5, 'gene_boost': 1}


In [7]:
results, aggregated = eval.evaluate(qrels, run_df)

In [8]:
aggregated

{'ndcg': 0.5338, 'Rprec': 0.2847, 'P_10': 0.48}

In [ ]:
run_df

In [ ]:
eval.experiment(topics_df, qrels)

In [ ]:
run_df = eval.run(topics_df)

In [ ]:
#run_df

In [ ]:
results, aggregated = eval.evaluate(qrels, run_df)

In [ ]:
aggregated